In [1]:
import numpy as np
import math
import torch
from torch import nn

import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt

from tempfile import TemporaryDirectory

from Models import BigGenerator
from Tools import KL, batchKL, sw, FunSW, FunKL

from tqdm import trange

from Experiments import get_setup

import itertools

from pandas import DataFrame
import pandas as pd

print(torch.__version__)


1.4.0


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
lat_dim=5

In [16]:
results_f=torch.load('Results/The_results-FuS.pt')
models=torch.load("Results/The_models.pt")

In [7]:
GeNmodels=torch.load("Results/GeNmodels.pt")

In [17]:
datasets=[d for d,i in models.items()]
datasets

['foong',
 'foong_sparse',
 'foong_mixed',
 'boston',
 'concrete',
 'energy',
 'powerplant',
 'wine',
 'yacht']

In [9]:
for d in datasets:
    models[d]['GeNNeVI']=GeNmodels[d]

In [10]:
torch.save(models, "Results/The_models.pt")

In [11]:
pd.DataFrame(results_f)

,"KL(G,HMC)","KL(F,HMC)","KL(HMC,G)","KL(HMC,F)","SW(HMC,G)","SW(HMC,F)","KL(G,G)","KL(F,F)","SW(G,G)","SW(F,F)"
foong,"(93.609, 3.109)","(53.885, 0.766)","(93.524, 4.952)","(39.348, 3.445)","(0.056, 0.001)","(0.058, 0.006)","(48.479, 17.907)","(20.715, 3.845)","(1.022, 0.028)","(0.766, 0.039)"
foong_sparse,"(74.764, 3.025)","(47.77, 6.073)","(40.717, 4.308)","(45.257, 3.863)","(0.019, 0.002)","(0.023, 0.005)","(39.101, 3.057)","(22.475, 4.131)","(0.841, 0.062)","(0.827, 0.039)"
boston,"(66.876, 2.614)","(3.713, 0.457)","(9.968, 0.375)","(66.41, 0.321)","(0.427, 0.002)","(1.543, 0.024)","(12.405, 3.185)","(2.973, 0.489)","(0.314, 0.003)","(3.152, 0.226)"
concrete,"(61.895, 1.988)","(3.092, 0.396)","(5.907, 0.21)","(61.223, 1.48)","(0.315, 0.004)","(0.963, 0.028)","(3.585, 0.841)","(2.861, 1.133)","(0.306, 0.016)","(3.665, 0.1)"
energy,"(44.847, 12.241)","(4.666, 0.39)","(12.987, 3.607)","(63.492, 0.493)","(1.077, 1.059)","(0.552, 0.006)","(34.873, 9.787)","(3.769, 0.457)","(0.408, 0.017)","(3.129, 0.038)"
powerplant,"(99.755, 1.941)","(10.611, 0.845)","(9.343, 0.337)","(84.8, 2.366)","(0.108, 0.0)","(0.604, 0.016)","(26.304, 13.344)","(7.367, 1.449)","(0.284, 0.002)","(4.619, 0.087)"
wine,"(79.141, 5.465)","(56.074, 4.187)","(20.131, 3.774)","(42.665, 0.464)","(0.623, 0.077)","(1.769, 0.059)","(63.66, 5.092)","(28.65, 3.819)","(0.848, 0.026)","(0.967, 0.057)"
yacht,"(97.657, 4.306)","(7.647, 1.742)","(14.405, 1.089)","(65.408, 2.995)","(0.238, 0.001)","(0.533, 0.027)","(13.611, 1.388)","(4.921, 2.939)","(0.313, 0.007)","(4.433, 0.103)"
foong_mixed,NaN,"(51.44, 6.024)",NaN,"(56.351, 9.219)",NaN,"(0.013, 0.003)",NaN,"(35.369, 7.566)",NaN,"(0.856, 0.085)"


In [12]:
metric='KL(G,HMC)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(s,t,projection,device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))

torch.save(results_f, 'Results/The_results-FuS.pt')

foong: 83.72640991210938
foong: 95.07848358154297
foong: 83.32372283935547
[83.72640991210938, 95.07848358154297, 83.32372283935547]
foong_sparse: 106.58514404296875
foong_sparse: 50.7448844909668
foong_sparse: 81.61347961425781
[106.58514404296875, 50.7448844909668, 81.61347961425781]
foong_mixed: 37.21516799926758
foong_mixed: 85.29702758789062
foong_mixed: inf
[37.21516799926758, 85.29702758789062, inf]


/home/yann/anaconda3/envs/myenv/lib/python3.8/site-packages/numpy/core/_methods.py:193: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


boston: 71.94464111328125
boston: 63.324806213378906
boston: 60.82177734375
[71.94464111328125, 63.324806213378906, 60.82177734375]
concrete: 63.389644622802734
concrete: 63.777435302734375
concrete: 56.08161163330078
[63.389644622802734, 63.777435302734375, 56.08161163330078]
energy: 31.717100143432617
energy: 40.113059997558594
energy: 45.105560302734375
[31.717100143432617, 40.113059997558594, 45.105560302734375]
powerplant: 78.12753295898438
powerplant: 88.77706146240234
powerplant: 82.40849304199219
[78.12753295898438, 88.77706146240234, 82.40849304199219]
wine: 138.62429809570312
wine: 152.37551879882812
wine: 165.37042236328125
[138.62429809570312, 152.37551879882812, 165.37042236328125]
yacht: 100.69210815429688
yacht: 88.57042694091797
yacht: 77.76911926269531
[100.69210815429688, 88.57042694091797, 77.76911926269531]


In [13]:
metric='KL(HMC,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(t,s,projection,device)
        if dataset == 'powerplant' or dataset == 'wine':
                K=FunKL(t,s,projection,device,k=2)

        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))

torch.save(results_f, 'Results/The_results-FuS.pt')

foong: 76.17546081542969
foong: 80.30618286132812
foong: 102.72454833984375
[76.17546081542969, 80.30618286132812, 102.72454833984375]
foong_sparse: 70.45323181152344
foong_sparse: 28.721288681030273
foong_sparse: 59.436012268066406
[70.45323181152344, 28.721288681030273, 59.436012268066406]
foong_mixed: 31.594911575317383
foong_mixed: 65.72838592529297
foong_mixed: 53.317440032958984
[31.594911575317383, 65.72838592529297, 53.317440032958984]
boston: 13.69332504272461
boston: 12.313982009887695
boston: 10.819339752197266
[13.69332504272461, 12.313982009887695, 10.819339752197266]
concrete: 6.929027557373047
concrete: 5.152300834655762
concrete: 4.3513102531433105
[6.929027557373047, 5.152300834655762, 4.3513102531433105]
energy: 4.787649631500244
energy: 18.117040634155273
energy: 9.41207218170166
[4.787649631500244, 18.117040634155273, 9.41207218170166]
powerplant: 4.470957279205322
powerplant: 5.868996620178223
powerplant: 4.792568206787109
[4.470957279205322, 5.868996620178223, 4.7

In [14]:
metric='SW(HMC,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunSW(t,s, projection, device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))

torch.save(results_f, 'Results/The_results-FuS.pt')

foong: 0.03863956034183502
foong: 0.041482649743556976
foong: 0.06014052778482437
[0.03863956034183502, 0.041482649743556976, 0.06014052778482437]
foong_sparse: 0.02043183520436287
foong_sparse: 0.012511606328189373
foong_sparse: 0.026751520112156868
[0.02043183520436287, 0.012511606328189373, 0.026751520112156868]
foong_mixed: 0.007612575776875019
foong_mixed: 0.012067067436873913
foong_mixed: 0.009420638903975487
[0.007612575776875019, 0.012067067436873913, 0.009420638903975487]
boston: 0.3555515706539154
boston: 0.352315217256546
boston: 0.3395988345146179
[0.3555515706539154, 0.352315217256546, 0.3395988345146179]
concrete: 0.24503858387470245
concrete: 0.23753103613853455
concrete: 0.23533733189105988
[0.24503858387470245, 0.23753103613853455, 0.23533733189105988]
energy: 0.11958160996437073
energy: 0.262752503156662
energy: 0.15333333611488342
[0.11958160996437073, 0.262752503156662, 0.15333333611488342]
powerplant: 0.06693074107170105
powerplant: 0.06832063943147659
powerplant: 

In [15]:
metric='KL(G,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        K=FunKL(t,s, projection,device)
        K_=FunKL(s,t, projection,device)
        print(dataset+': '+str((K.item(), K_.item())))
        KLs.append(K.item())
        KLs.append(K_.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))

torch.save(results_f, 'Results/The_results-FuS.pt')

foong: (51.41547775268555, 50.66993713378906)
foong: (113.11947631835938, 120.60809326171875)
foong: (139.20404052734375, 143.68048095703125)
[51.41547775268555, 50.66993713378906, 113.11947631835938, 120.60809326171875, 139.20404052734375, 143.68048095703125]
foong_sparse: (85.49947357177734, 105.23858642578125)
foong_sparse: (91.98794555664062, 124.12821197509766)
foong_sparse: (72.67162322998047, 76.69781494140625)
[85.49947357177734, 105.23858642578125, 91.98794555664062, 124.12821197509766, 72.67162322998047, 76.69781494140625]
foong_mixed: (105.54366302490234, 81.10282897949219)
foong_mixed: (87.2176284790039, 76.01868438720703)
foong_mixed: (41.0399169921875, 35.22309875488281)
[105.54366302490234, 81.10282897949219, 87.2176284790039, 76.01868438720703, 41.0399169921875, 35.22309875488281]
boston: (10.14293098449707, 16.810579299926758)
boston: (10.299999237060547, 16.613859176635742)
boston: (13.634408950805664, 12.495408058166504)
[10.14293098449707, 16.810579299926758, 10.299

In [19]:
metric='SW(G,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    SWs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(5000).detach()
        G.load_state_dict(n)
        t=G(5000).detach()
        SW=sw(s.cpu(),t.cpu(),'cpu')
        print(SW)
        SWs.append(SW.item())
    
    results_f[metric][dataset]=(np.mean(SWs).round(decimals=3), np.std(SWs).round(decimals=3))

torch.save(results_f, 'Results/The_results-FuS.pt')

1.145926387801447
1.1731283709247438
1.1682749379019433
2.1196204765484445
2.0088715695049215
1.1350872800544358
1.9999813706183827
2.2950951781593782
1.3314184473949104
0.31673511607653837
0.32724751263097673
0.3375390045226814
0.3277733655463263
0.4280897163050753
0.42255116971828455
0.3998225612738263
0.3977146120425136
0.3992089593285082
0.28099026787559755
0.29413767543185526
0.2897042512156441
1.2193768920380235
1.1472564484155732
1.2023013938979865
0.3265187720348732
0.32297317851687846
0.3109375207105306


In [35]:
torch.save(results_f, 'Results/The_results-FuS.pt')
pd.DataFrame(results_f,index=datasets)
#(1.755, 0.44)
#torch.save(Df.to_latex(),'Results/results-FuS.tex')

,"KL(G,HMC)","KL(F,HMC)","KL(HMC,G)","KL(HMC,F)","SW(HMC,G)","SW(HMC,F)","KL(G,G)","KL(F,F)","SW(G,G)","SW(F,F)"
foong,"(87.376, 5.449)","(53.885, 0.766)","(86.402, 11.664)","(39.348, 3.445)","(0.047, 0.01)","(0.058, 0.006)","(103.116, 38.248)","(20.715, 3.845)","(1.162, 0.012)","(0.766, 0.039)"
foong_sparse,"(79.648, 22.839)","(47.77, 6.073)","(52.87, 17.658)","(45.257, 3.863)","(0.02, 0.006)","(0.023, 0.005)","(92.704, 17.581)","(22.475, 4.131)","(1.755, 0.44)","(0.827, 0.039)"
foong_mixed,"(62.062, 21.335)","(51.44, 6.024)","(50.214, 14.107)","(56.351, 9.219)","(0.01, 0.002)","(0.013, 0.003)","(71.024, 25.037)","(35.369, 7.566)","(1.924, 0.409)","(0.856, 0.085)"
boston,"(65.364, 4.764)","(3.713, 0.457)","(12.276, 1.174)","(66.41, 0.321)","(0.349, 0.007)","(1.543, 0.024)","(13.333, 2.678)","(2.973, 0.489)","(0.332, 0.005)","(3.152, 0.226)"
concrete,"(61.083, 3.54)","(3.092, 0.396)","(5.478, 1.077)","(61.223, 1.48)","(0.239, 0.004)","(0.963, 0.028)","(6.456, 1.911)","(2.861, 1.133)","(0.393, 0.046)","(3.665, 0.1)"
energy,"(38.979, 5.524)","(4.666, 0.39)","(10.772, 5.526)","(63.492, 0.493)","(0.179, 0.061)","(0.552, 0.006)","(38.578, 5.988)","(3.769, 0.457)","(0.399, 0.001)","(3.129, 0.038)"
powerplant,"(83.104, 4.375)","(10.611, 0.845)","(5.044, 0.598)","(84.8, 2.366)","(0.068, 0.001)","(0.604, 0.016)","(17.025, 2.134)","(7.367, 1.449)","(0.288, 0.005)","(4.619, 0.087)"
wine,"(152.123, 10.921)","(56.074, 4.187)","(45.575, 4.928)","(42.665, 0.464)","(0.871, 0.08)","(1.769, 0.059)","(165.476, 9.694)","(28.65, 3.819)","(1.19, 0.031)","(0.967, 0.057)"
yacht,"(89.011, 9.363)","(7.647, 1.742)","(11.142, 0.915)","(65.408, 2.995)","(0.151, 0.007)","(0.533, 0.027)","(13.432, 4.219)","(4.921, 2.939)","(0.32, 0.007)","(4.433, 0.103)"


In [26]:
metric='KL(G,HMC)'
dataset='foong_mixed'
method='GeNNeVI'

setup_ = get_setup(dataset)
setup=setup_.Setup(device) 
G=BigGenerator(lat_dim, setup.param_count, device).to(device)
models_pairs=list(itertools.combinations(models[dataset][method].items(),2))
def projection(t,s,m):
    return setup.projection(t,s,m,ratio_ood=1)

KLs=[]
for i,m in models[dataset][method].items():
    t=models[dataset]['HMC'][0].to(device)
    G.load_state_dict(m)
    s=G(t.shape[0]).detach()
    K=FunKL(s,t,projection,device,k=2)
    print(dataset+': '+str(K.item()))
    KLs.append(K.item())
print(KLs)
results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))

torch.save(results_f, 'Results/The_results-FuS.pt')

foong_mixed: 32.435672760009766
foong_mixed: 81.82311248779297
foong_mixed: 71.92577362060547
[32.435672760009766, 81.82311248779297, 71.92577362060547]


In [30]:
metric='SW(G,G)'
dataset='boston'
method='GeNNeVI'

setup_ = get_setup(dataset)
setup=setup_.Setup(device) 
G=BigGenerator(lat_dim, setup.param_count, device).to(device)
models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

SWs=[]
for (i,m),(j,n) in models_pairs:
    G.load_state_dict(m)
    s=G(5000).detach()
    G.load_state_dict(n)
    t=G(5000).detach()
    SW=sw(s.cpu(),t.cpu(),'cpu')
    print(SW)
    SWs.append(SW.item())

results_f[metric][dataset]=(np.mean(SWs).round(decimals=3), np.std(SWs).round(decimals=3))

torch.save(results_f, 'Results/The_results-FuS.pt')

0.33115247421401817
0.3262315659856364
0.33883109738379


In [3]:
HMC=torch.load('Results/models_HMC.pt')

In [4]:
dict_m=torch.load('Results/HMC_metrics.pt')
pd.DataFrame(dict_m).transpose()

,RMSE,LPP,PICP,MPIW
boston,[2.61],-1.882,0.825,25.958
concrete,[7.473],-2.447,0.828,57.155
energy,[2.628],-1.301,0.393,13.387
foong,1.154,-2.502,0.187,0.954
powerplant,[3.88],-2.055,0.229,13.011
yacht,[3.828],-1.326,0.728,29.559
wine,[0.585],-1.107,0.452,1.204
foong_sparse,0.319,-0.954,0.144,0.315
foong_mixed,0.278,-0.491,0.095,0.198


In [6]:
dataset='boston'
setup_ = get_setup(dataset)
setup=setup_.Setup(device) 
    

log_device = 'cpu'
theta_ = HMC[dataset]
theta = theta_

LPP_test, RMSE_test, _, PICP_test, MPIW_test = setup.evaluate_metrics(theta,log_device)

torch.Size([1])


In [27]:
LPP_test, RMSE_test, _, PICP_test, MPIW_test

((tensor(-2.5020), tensor(4.6867)),
 (tensor(1.1541), tensor(1.5386)),
 (tensor(0.1023), tensor(0.1228)),
 tensor(0.1871),
 tensor(0.9541))

In [25]:
for d in datasets:
    print(d+': '+str(HMC[d].shape))

foong: torch.Size([9999, 151])
foong_sparse: torch.Size([9999, 151])
foong_mixed: torch.Size([8933, 151])
boston: torch.Size([10000, 751])
concrete: torch.Size([10000, 501])
energy: torch.Size([9999, 501])
powerplant: torch.Size([2222, 301])
wine: torch.Size([8720, 651])
yacht: torch.Size([9999, 401])


In [7]:
torch.tensor(0.).shape

torch.Size([])